In [76]:
!git pull

remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 8 (delta 2), reused 8 (delta 2), pack-reused 0
Unpacking objects: 100% (8/8), done.
From https://github.com/basilwong/awstest1
   9a15831..70308c6  master     -> origin/master
Updating 9a15831..70308c6
Fast-forward
 ...dio_file_1586688313.4434075.mp3_accompaniment.wav | Bin 0 -> 10584058 bytes
 ...dio_file_1586688359.6707158.mp3_accompaniment.wav | Bin 0 -> 10584058 bytes
 ...udio_file_1586688451.638458.mp3_accompaniment.wav | Bin 0 -> 10584058 bytes
 ...dio_file_1586688129.2001228.mp3_accompaniment.wav | Bin 0 -> 10584058 bytes
 ...udio_file_1586688267.341563.mp3_accompaniment.wav | Bin 0 -> 10584058 bytes
 ...udio_file_1586688221.484059.mp3_accompaniment.wav | Bin 0 -> 10584058 bytes
 ...dio_file_1586688175.3730624.mp3_accompaniment.wav | Bin 0 -> 10584058 bytes
 ...dio_file_1586688497.7542431.mp3_accompaniment.wav | Bin 0 -> 2523634 bytes


### Add Dependencies

In [1]:
import sagemaker as sage
from sagemaker import get_execution_role

import zipfile
import os

from sagemaker import ModelPackage


# some_file.py
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.append('src')

!pip install pydub

import audio_util
import processing_util

You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pydub/utils.py:165: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
# Execution role
role = get_execution_role()
# S3 prefixes
common_prefix = "source_separation"
batch_inference_input_prefix = common_prefix + "/batch-inference-input-data"
# Sagemaker Session
sagemaker_session = sage.Session()
# Arn for Source Separator Model Package
modelpackage_arn = 'arn:aws:sagemaker:us-east-2:057799348421:model-package/source-separation-v11570291536-75ed8128ecee95e142ec4404d884ecad'



### Creating the Model

In [3]:
from sagemaker import ModelPackage

def predict_wrapper(endpoint, session):
    return sage.RealTimePredictor(endpoint, session, content_type='application/x-recordio-protobuf')

model = ModelPackage(role=role,
                     model_package_arn=modelpackage_arn,
                     sagemaker_session=sagemaker_session,
                     predictor_cls=predict_wrapper)

### Running the Batch Job

Note that if the initial audio file is longer than around 30 seconds, it is too large for the model. The split_mp3() method in  src.audio_util works around this by splitting an mp3 file into 30 second segments. 

In [ ]:
audio_util.split_mp3("../songs/drake-toosie_slide.mp3", "../source-separation-input/")

In [4]:
batch_input_folder = "source-separation-input"


transform_input = sagemaker_session.upload_data(batch_input_folder, key_prefix=batch_inference_input_prefix)
print("Transform input uploaded to " + transform_input)

Transform input uploaded to s3://sagemaker-us-east-2-075178354542/source_separation/batch-inference-input-data


In [5]:
import json 
import uuid

bucket = sagemaker_session.default_bucket()

transformer = model.transformer(1, 'ml.m4.xlarge', strategy='SingleRecord', output_path='s3://'+bucket+'/'+common_prefix+'/batch-transform-output')
transformer.transform(transform_input, content_type='application/x-recordio-protobuf')
transformer.wait()

print("Batch Transform output saved to " + transformer.output_path)

....................Starting the inference server with 4 workers.
[2020-04-13 04:48:15 +0000] [10] [INFO] Starting gunicorn 19.9.0
[2020-04-13 04:48:15 +0000] [10] [INFO] Listening at: unix:/tmp/gunicorn.sock (10)
[2020-04-13 04:48:15 +0000] [10] [INFO] Using worker: gevent
[2020-04-13 04:48:15 +0000] [14] [INFO] Booting worker with pid: 14
[2020-04-13 04:48:15 +0000] [15] [INFO] Booting worker with pid: 15
[2020-04-13 04:48:15 +0000] [16] [INFO] Booting worker with pid: 16
[2020-04-13 04:48:15 +0000] [17] [INFO] Booting worker with pid: 17
Testing...
Testing...
2020-04-13 04:48:39.079016: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
169.254.255.130 - - [13/Apr/2020:04:48:39 +0000] "GET /ping HTTP/1.1" 200 1 "-" "Go-http-client/1.1"
169.254.255.130 - - [13/Apr/2020:04:48:39 +0000] "GET /execution-parameters HTTP/1.1" 404 2 "-" "Go-http-client/1.1"
Input path : /tmp/audio_file_1586753319

['audio_file_1586753595.1332912.mp3_vocals.wav', 'audio_file_1586753595.1332912.mp3_accompaniment.wav']
['audio_file_1586753595.1332912.mp3_vocals.wav', 'audio_file_1586753595.1332912.mp3_accompaniment.wav']
169.254.255.130 - - [13/Apr/2020:04:54:01 +0000] "POST /invocations HTTP/1.1" 200 19459102 "-" "Go-http-client/1.1"
Input path : /tmp/audio_file_1586753641.1667993.mp3
Producing source estimates for input mixture file /tmp/audio_file_1586753641.1667993.mp3
Testing...
169.254.255.130 - - [13/Apr/2020:04:54:01 +0000] "POST /invocations HTTP/1.1" 200 19459102 "-" "Go-http-client/1.1"
Input path : /tmp/audio_file_1586753641.1667993.mp3
Producing source estimates for input mixture file /tmp/audio_file_1586753641.1667993.mp3
Testing...
Num of variables64
Pre-trained model restored for song prediction
Num of variables64
Pre-trained model restored for song prediction
169.254.255.130 - - [13/Apr/2020:04:54:47 +0000] "POST /invocations HTTP/1.1" 200 19035684 "-" "Go-http-client/1.1"
Input pa

### Processing the Batch Output

In [7]:
import boto3
# Downloading files from s3.
s3 = boto3.resource('s3')
my_bucket = s3.Bucket(sagemaker_session.default_bucket())
prefix = "source_separation/batch-transform-output/"
i = 0
audio_util.clear_folder('source-separation-output/batch-transform-output')
for object_summary in my_bucket.objects.filter(Prefix=prefix):
    i = i + 1
    file_name = object_summary.key.split('/')[-1]
    print(file_name)
    my_bucket.download_file(prefix+ file_name, 'source-separation-output/batch-transform-output/output-{}.zip'.format(i))

input1.mp3.out
input2.mp3.out
input3.mp3.out
input4.mp3.out
input5.mp3.out
input6.mp3.out
input7.mp3.out
input8.mp3.out
input9.mp3.out


In [29]:
# Extracting files from zip files. 
audio_util.clear_folder('source-separation-output/extracted')
for file in os.listdir('source-separation-output/batch-transform-output'):
    print(file)
    with zipfile.ZipFile('source-separation-output/batch-transform-output/'+file, 'r') as zip_ref:
        zip_ref.extractall('source-separation-output/extracted/'+file.split('.')[0]+'/')

output-3.zip
output-6.zip
output-8.zip
output-9.zip
output-1.zip
output-4.zip
output-7.zip
output-2.zip
output-5.zip


In [31]:
import os
# Separating the vocal files and the background sound files.
audio_util.clear_folder('source-separation-output/vocals')
audio_util.clear_folder('source-separation-output/background')
for i, folder in enumerate(sorted(os.listdir('source-separation-output/extracted/'))):
    for file in os.listdir('source-separation-output/extracted/' + folder + '/output'):
        new_file_name = str(i).zfill(5)
        print(new_file_name)
        if "vocals" in file:
            os.rename('source-separation-output/extracted/' + folder + '/output/' + file, 'source-separation-output/vocals/' + new_file_name)
        elif "accompaniment" in file:
            os.rename('source-separation-output/extracted/' + folder + '/output/' + file, 'source-separation-output/background/' + new_file_name)

0audio_file_1586753319.4046214.mp3_accompaniment.wav
0audio_file_1586753319.4046214.mp3_vocals.wav
1audio_file_1586753365.3933253.mp3_accompaniment.wav
1audio_file_1586753365.3933253.mp3_vocals.wav
2audio_file_1586753411.4280498.mp3_vocals.wav
2audio_file_1586753411.4280498.mp3_accompaniment.wav
3audio_file_1586753457.4691133.mp3_vocals.wav
3audio_file_1586753457.4691133.mp3_accompaniment.wav
4audio_file_1586753503.4803622.mp3_vocals.wav
4audio_file_1586753503.4803622.mp3_accompaniment.wav
5audio_file_1586753549.2771432.mp3_vocals.wav
5audio_file_1586753549.2771432.mp3_accompaniment.wav
6audio_file_1586753595.1332912.mp3_vocals.wav
6audio_file_1586753595.1332912.mp3_accompaniment.wav
7audio_file_1586753641.1667993.mp3_accompaniment.wav
7audio_file_1586753641.1667993.mp3_vocals.wav
8audio_file_1586753687.3142326.mp3_accompaniment.wav
8audio_file_1586753687.3142326.mp3_vocals.wav


### Transcribe the Vocal Files

In [32]:
# Upload the Vocal files onto s3
local_vocals_folder = "source-separation-output/vocals/"
transcribe_input_prefix = "transcribe-input"

transcribe_input = sagemaker_session.upload_data(local_vocals_folder, key_prefix=transcribe_input_prefix)
print("Transcribe input uploaded to " + transcribe_input)

Transcribe input uploaded to s3://sagemaker-us-east-2-075178354542/transcribe-input


In [75]:
# Start a transcription job for each file. Add the transcription to finsihed jobs once finished. 

import boto3
from datetime import datetime

transcribe = boto3.client('transcribe')
output_bucket_name = "transcribe-output"
uri_prefix = "https://sagemaker-us-east-2-075178354542.s3.us-east-2.amazonaws.com/transcribe-input/"
finished_jobs = list()

for file in sorted(os.listdir(local_vocals_folder)):

    print("Transcribing: " + file)
    job_uri = uri_prefix + file
    transcribe.start_transcription_job(
        TranscriptionJobName=file,
        Media={'MediaFileUri': job_uri},
        MediaFormat='wav',
        LanguageCode='en-US'
    )
    while True:
        status = transcribe.get_transcription_job(TranscriptionJobName=file)
        if status['TranscriptionJob']['TranscriptionJobStatus'] in ['COMPLETED', 'FAILED']:
            break
    
    api_data = requests.get(url=status['TranscriptionJob']['Transcript']['TranscriptFileUri'])
    data = api_data.json()
    finished_jobs.append(data)
    dump_file_name = 'transcribe-output/transcription' + file.split("audio")[0] + '.json'
    # Writing to json files for analysis purposes.
    with open(dump_file_name, 'w') as f:
        json.dump(data, f, indent=4)
    transcribe.delete_transcription_job(TranscriptionJobName=file)
    
finished_jobs.sort(key=lambda x : x['jobName'])

Transcribing: 0audio_file_1586753319.4046214.mp3_vocals.wav
Transcribing: 1audio_file_1586753365.3933253.mp3_vocals.wav
Transcribing: 2audio_file_1586753411.4280498.mp3_vocals.wav
Transcribing: 3audio_file_1586753457.4691133.mp3_vocals.wav
Transcribing: 4audio_file_1586753503.4803622.mp3_vocals.wav
Transcribing: 5audio_file_1586753549.2771432.mp3_vocals.wav
Transcribing: 6audio_file_1586753595.1332912.mp3_vocals.wav
Transcribing: 7audio_file_1586753641.1667993.mp3_vocals.wav
Transcribing: 8audio_file_1586753687.3142326.mp3_vocals.wav


In [74]:
transcribe.delete_transcription_job(TranscriptionJobName="0audio_file_1586753319.4046214.mp3_vocals.wav")

{'ResponseMetadata': {'RequestId': '0caac80b-36eb-4400-bb48-87638b55d504',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Mon, 13 Apr 2020 09:54:13 GMT',
   'x-amzn-requestid': '0caac80b-36eb-4400-bb48-87638b55d504',
   'content-length': '0',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

### Giving Transcriptions to Amazon Polly

In [ ]:

transcribe_output_folder = "transcribe_output/"
index = 0

for file in sorted(os.listdir(transcribe_output_folder)):
    
    transcription_batch = processing_util.load_json(transcribe_output_folder + file)
    expected_start_time = 0
    

Mixing Audio:

https://stackoverflow.com/questions/7629873/how-do-i-mix-audio-files-using-python

Pitch Modulation:

https://stackoverflow.com/questions/38923438/does-pydub-support-pitch-modulation

